In [6]:
#In Jupyter notebooks, you will need to run this command before doing any plotting
%matplotlib inline
from sutils import *
import os, json
from glob import glob
import Augmentor
import keras
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model,load_model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.layers.core import  Flatten, Dropout, Lambda

from keras.layers import Conv2D

from keras import backend as K

In [7]:
limit_gpu_mem()

In [39]:
#path = "data/dogscats/"
#path = "data/dogscats/sample/"
path ='data/brandshoes/'
#path ='D:/Datascience_lessons/raw data/MovieGenre/images_filtered/'

In [48]:
batch_size=25
no_of_epochs=15

In [45]:
# Prepare images for training in batches
# NB: They must be in subdirectories named based on their category


batches = get_batches(path+'train', batch_size=batch_size)
val_batches = get_batches(path+'valid', batch_size=batch_size*2)

Found 113 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


This will download the Inception weights from
https://github.com/fchollet/deep-learning-models/releases/download/v0.5/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

In [11]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

In [7]:
base_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, None, None, 32 864         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, None, None, 32 96          conv2d_1[0][0]                   
____________________________________________________________________________________________________
activation_1 (Activation)        (None, None, None, 32 0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

activation_12 (Activation)       (None, None, None, 32 0           batch_normalization_12[0][0]     
____________________________________________________________________________________________________
mixed0 (Concatenate)             (None, None, None, 25 0           activation_6[0][0]               
                                                                   activation_8[0][0]               
                                                                   activation_11[0][0]              
                                                                   activation_12[0][0]              
____________________________________________________________________________________________________
conv2d_16 (Conv2D)               (None, None, None, 64 16384       mixed0[0][0]                     
____________________________________________________________________________________________________
batch_normalization_16 (BatchNor (None, None, None, 64 192         conv2d_16[0][0]         

____________________________________________________________________________________________________
conv2d_26 (Conv2D)               (None, None, None, 64 18432       average_pooling2d_3[0][0]        
____________________________________________________________________________________________________
batch_normalization_20 (BatchNor (None, None, None, 64 192         conv2d_20[0][0]                  
____________________________________________________________________________________________________
batch_normalization_22 (BatchNor (None, None, None, 64 192         conv2d_22[0][0]                  
____________________________________________________________________________________________________
batch_normalization_25 (BatchNor (None, None, None, 96 288         conv2d_25[0][0]                  
____________________________________________________________________________________________________
batch_normalization_26 (BatchNor (None, None, None, 64 192         conv2d_26[0][0]         

____________________________________________________________________________________________________
batch_normalization_33 (BatchNor (None, None, None, 12 384         conv2d_33[0][0]                  
____________________________________________________________________________________________________
batch_normalization_38 (BatchNor (None, None, None, 12 384         conv2d_38[0][0]                  
____________________________________________________________________________________________________
activation_33 (Activation)       (None, None, None, 12 0           batch_normalization_33[0][0]     
____________________________________________________________________________________________________
activation_38 (Activation)       (None, None, None, 12 0           batch_normalization_38[0][0]     
____________________________________________________________________________________________________
average_pooling2d_4 (AveragePool (None, None, None, 76 0           mixed3[0][0]            

____________________________________________________________________________________________________
conv2d_49 (Conv2D)               (None, None, None, 19 215040      activation_48[0][0]              
____________________________________________________________________________________________________
conv2d_50 (Conv2D)               (None, None, None, 19 147456      average_pooling2d_5[0][0]        
____________________________________________________________________________________________________
batch_normalization_41 (BatchNor (None, None, None, 19 576         conv2d_41[0][0]                  
____________________________________________________________________________________________________
batch_normalization_44 (BatchNor (None, None, None, 19 576         conv2d_44[0][0]                  
____________________________________________________________________________________________________
batch_normalization_49 (BatchNor (None, None, None, 19 576         conv2d_49[0][0]         

____________________________________________________________________________________________________
activation_54 (Activation)       (None, None, None, 19 0           batch_normalization_54[0][0]     
____________________________________________________________________________________________________
activation_59 (Activation)       (None, None, None, 19 0           batch_normalization_59[0][0]     
____________________________________________________________________________________________________
activation_60 (Activation)       (None, None, None, 19 0           batch_normalization_60[0][0]     
____________________________________________________________________________________________________
mixed6 (Concatenate)             (None, None, None, 76 0           activation_51[0][0]              
                                                                   activation_54[0][0]              
                                                                   activation_59[0][0]     

batch_normalization_73 (BatchNor (None, None, None, 19 576         conv2d_73[0][0]                  
____________________________________________________________________________________________________
activation_73 (Activation)       (None, None, None, 19 0           batch_normalization_73[0][0]     
____________________________________________________________________________________________________
conv2d_74 (Conv2D)               (None, None, None, 19 258048      activation_73[0][0]              
____________________________________________________________________________________________________
batch_normalization_74 (BatchNor (None, None, None, 19 576         conv2d_74[0][0]                  
____________________________________________________________________________________________________
activation_74 (Activation)       (None, None, None, 19 0           batch_normalization_74[0][0]     
___________________________________________________________________________________________

____________________________________________________________________________________________________
activation_79 (Activation)       (None, None, None, 38 0           batch_normalization_79[0][0]     
____________________________________________________________________________________________________
activation_80 (Activation)       (None, None, None, 38 0           batch_normalization_80[0][0]     
____________________________________________________________________________________________________
activation_83 (Activation)       (None, None, None, 38 0           batch_normalization_83[0][0]     
____________________________________________________________________________________________________
activation_84 (Activation)       (None, None, None, 38 0           batch_normalization_84[0][0]     
____________________________________________________________________________________________________
batch_normalization_85 (BatchNor (None, None, None, 19 576         conv2d_85[0][0]         

                                                                   activation_89[0][0]              
____________________________________________________________________________________________________
concatenate_2 (Concatenate)      (None, None, None, 76 0           activation_92[0][0]              
                                                                   activation_93[0][0]              
____________________________________________________________________________________________________
activation_94 (Activation)       (None, None, None, 19 0           batch_normalization_94[0][0]     
____________________________________________________________________________________________________
mixed10 (Concatenate)            (None, None, None, 20 0           activation_86[0][0]              
                                                                   mixed9_1[0][0]                   
                                                                   concatenate_2[0][0]     

In [12]:
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# let's add a fully-connected layer

x = Dense(1024, activation='relu')(x)

x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(2, activation='softmax')(x)

In [13]:
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [14]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    print(layer.name)
    layer.trainable = False

input_1
conv2d_1
batch_normalization_1
activation_1
conv2d_2
batch_normalization_2
activation_2
conv2d_3
batch_normalization_3
activation_3
max_pooling2d_1
conv2d_4
batch_normalization_4
activation_4
conv2d_5
batch_normalization_5
activation_5
max_pooling2d_2
conv2d_9
batch_normalization_9
activation_9
conv2d_7
conv2d_10
batch_normalization_7
batch_normalization_10
activation_7
activation_10
average_pooling2d_1
conv2d_6
conv2d_8
conv2d_11
conv2d_12
batch_normalization_6
batch_normalization_8
batch_normalization_11
batch_normalization_12
activation_6
activation_8
activation_11
activation_12
mixed0
conv2d_16
batch_normalization_16
activation_16
conv2d_14
conv2d_17
batch_normalization_14
batch_normalization_17
activation_14
activation_17
average_pooling2d_2
conv2d_13
conv2d_15
conv2d_18
conv2d_19
batch_normalization_13
batch_normalization_15
batch_normalization_18
batch_normalization_19
activation_13
activation_15
activation_18
activation_19
mixed1
conv2d_23
batch_normalization_23
activat

In [11]:
for layer in model.layers:
    print(layer.name)
    print(layer.trainable)

input_1
False
conv2d_1
False
batch_normalization_1
False
activation_1
False
conv2d_2
False
batch_normalization_2
False
activation_2
False
conv2d_3
False
batch_normalization_3
False
activation_3
False
max_pooling2d_1
False
conv2d_4
False
batch_normalization_4
False
activation_4
False
conv2d_5
False
batch_normalization_5
False
activation_5
False
max_pooling2d_2
False
conv2d_9
False
batch_normalization_9
False
activation_9
False
conv2d_7
False
conv2d_10
False
batch_normalization_7
False
batch_normalization_10
False
activation_7
False
activation_10
False
average_pooling2d_1
False
conv2d_6
False
conv2d_8
False
conv2d_11
False
conv2d_12
False
batch_normalization_6
False
batch_normalization_8
False
batch_normalization_11
False
batch_normalization_12
False
activation_6
False
activation_8
False
activation_11
False
activation_12
False
mixed0
False
conv2d_16
False
batch_normalization_16
False
activation_16
False
conv2d_14
False
conv2d_17
False
batch_normalization_14
False
batch_normalization_17
F

In [17]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [50]:
hist = model.fit_generator(batches, steps_per_epoch=8, epochs=no_of_epochs,verbose=1,
               validation_data=val_batches, validation_steps=3)


Epoch 1/15
8/8 [==============================] - ETA: 2s - loss: 0.0109 - acc: 1.000 - ETA: 1s - loss: 0.0220 - acc: 1.000 - ETA: 1s - loss: 0.0194 - acc: 1.000 - ETA: 1s - loss: 0.0939 - acc: 0.950 - ETA: 0s - loss: 0.0841 - acc: 0.960 - ETA: 0s - loss: 0.0780 - acc: 0.960 - ETA: 0s - loss: 0.0761 - acc: 0.965 - 2s - loss: 0.0878 - acc: 0.9645 - val_loss: 0.2206 - val_acc: 0.8750
Epoch 2/15
8/8 [==============================] - ETA: 2s - loss: 0.0154 - acc: 1.000 - ETA: 1s - loss: 0.0190 - acc: 1.000 - ETA: 1s - loss: 0.0198 - acc: 1.000 - ETA: 1s - loss: 0.0461 - acc: 1.000 - ETA: 0s - loss: 0.0519 - acc: 1.000 - ETA: 0s - loss: 0.0442 - acc: 1.000 - ETA: 0s - loss: 0.0391 - acc: 1.000 - 3s - loss: 0.0345 - acc: 1.0000 - val_loss: 0.2609 - val_acc: 0.8750
Epoch 3/15
8/8 [==============================] - ETA: 1s - loss: 0.0458 - acc: 1.000 - ETA: 1s - loss: 0.0239 - acc: 1.000 - ETA: 1s - loss: 0.0178 - acc: 1.000 - ETA: 1s - loss: 0.0139 - acc: 1.000 - ETA: 0s - loss: 0.0114 - acc

In [51]:
metrics = model.evaluate_generator(val_batches,10,10,workers=1,pickle_safe=False)
print("model accuracy:",metrics[1])

model accuracy: 0.9375


In [52]:
model.save('brandedshoes.hdf5')

In [53]:
model = load_model('brandedshoes.hdf5')

In [54]:
image_path ='data/brandshoes/test/'

In [65]:
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
import numpy as np

img_path = os.path.join(image_path, 'sb-check-solarsoft-canvas-skateboarding-shoe.jpg')
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
print('Input image shape:', x.shape)

Input image shape: (1, 224, 224, 3)


In [66]:
preds = model.predict(x)
print('Predicted:', preds)

Predicted: [[ 0.33529824  0.66470176]]


In [67]:
result= np.argmax(preds)
if result==0:
    print("Its a Addidas")
else:
    print("Its a Nike")

Its a Nike
